In [1]:
text = 'this text is a test, i am sick my dog is sick'

In [2]:
from keybert import KeyBERT
keybert = KeyBERT('models/nlp/semi_supervised/model')
keywords = keybert.extract_keywords(
    text,
    keyphrase_ngram_range=(1, 2),
    stop_words="english",
    use_maxsum=True,
    nr_candidates=10,
    top_n=5,
    use_mmr=True,
)


No sentence-transformers model found with name models/nlp/semi_supervised/model. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at models/nlp/semi_supervised/model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# import transformers
import transformers
from transformers import pipeline
hf_model = pipeline('feature-extraction', model='models/nlp/semi_supervised/model')
kw_model = KeyBERT(model=hf_model)
keywords = kw_model.extract_keywords(
    text,
    keyphrase_ngram_range=(1, 2),
    stop_words="english",
    use_maxsum=True,
    nr_candidates=10,
    top_n=5,
    use_mmr=True,
)
keywords

Some weights of the model checkpoint at models/nlp/semi_supervised/model were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[('test sick', 0.6507),
 ('sick dog', 0.6467),
 ('sick', 0.5757),
 ('text test', 0.5491),
 ('text', 0.3682)]